In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import gc
import os
PATH_DATA = './data/'

In [ ]:
import joblib
training = joblib.load('./model/training.pkl')

In [ ]:
train_neg_sample = training.loc[training['click']==0]
train_neg_sample1  = train_neg_sample.sample(n=3000000*2)
train_pos_sample = training.loc[training['click']!=0]
train_pos_sample1 = train_pos_sample.sample(n=3000000)

In [ ]:
train = pd.concat([train_neg_sample1,train_pos_sample1]).reset_index(drop=True)
train['type']=0
train.dtypes
train

In [ ]:
train['click'].value_counts()

In [ ]:
train['impression'].sum()

In [ ]:
train.loc[train['click'] > 0, 'click'] = 1
train

In [ ]:
train['click'].value_counts()

In [ ]:
num = train['impression'].sum()

In [ ]:
def convert(train_array):
    convert_feature = np.zeros(shape=(num,13))
    index = 0
    for i in range(len(train_array)):
        row = train_array[i]
        #print(row)
        num_of_pos = int(row[0])
        for times in range(int(row[1])):
            if num_of_pos !=0:
                row[12] = int(1)
                convert_feature[index,:] = row
                num_of_pos = num_of_pos - 1
            else:
                row[12] = int(0)
                convert_feature[index,:] = row
            index = index + 1
    return convert_feature
train_array = train.values
train_data= convert(train_array)

In [ ]:
import pandas as pd
columns = ['click','impression', 'displayurl', 'adid', 'advertiserid', 'depth', 
                     'position', 'queryid', 'keywordid', 'titleid', 'descriptionid', 'userid','type']
train_new = pd.DataFrame(train_data, columns=list(columns),dtype=np.int64)
train_new[['advertiserid']] = train_new[['advertiserid']].astype('uint64')
train_new

In [ ]:
train_new = train_new.drop_duplicates(subset=None, keep='first', inplace=False)

In [ ]:
train_new.shape

In [ ]:
train_new['type'].value_counts()

In [ ]:
import os
PATH_DATA = './model/'
test = None
if os.path.exists(PATH_DATA + 'test.pkl'):
    test = pd.read_pickle(PATH_DATA + 'test.pkl')
else:
    test = pd.read_csv(PATH_ABOVE + 'test.txt', header=None, sep='\t', nrows=None)
    test.columns = ['displayurl', 'adid', 'advertiserid', 'depth', 'position',
                    'queryid', 'keywordid', 'titleid', 'descriptionid', 'userid']
    test.to_pickle(PATH_DATA + 'test.pkl')
test = test[['adid', 'position', 'queryid', 'keywordid', 'userid']]
test

In [ ]:
X = train_new[['adid', 'position', 'queryid', 'keywordid', 'userid']]
y = train_new[['click','impression','type']]

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

In [ ]:
train_test=pd.concat([X,test])
train_test

In [ ]:
enc.fit(train_test)
train_onehot = enc.transform(X)
test_onehot = enc.transform(test)

In [ ]:
del train
del test
gc.collect()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_onehot, y, test_size=0.1, random_state=42)

In [ ]:
clf = MultinomialNB(0.7)
clf.fit(X_train, y_train['type'])

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
y_test['click'].tolist()

In [ ]:
y_test['impression'].tolist()

In [ ]:
y_pre = clf.predict_proba(X_test)

In [ ]:
prob = y_pre[:,1]
prob

In [ ]:
auc = scoreClickAUC(y_test['click'].tolist(), y_test['impression'].tolist(), prob)
auc

In [ ]:
prob_pre = clf.predict_proba(test_onehot)

In [ ]:
y_pre_all = prob_pre[:, 1]

In [ ]:
def scoreClickAUC(num_clicks, num_impressions, predicted_ctr):

    i_sorted = sorted(range(len(predicted_ctr)),key=lambda i: predicted_ctr[i],
                      reverse=True)
    auc_temp = 0.0
    click_sum = 0.0
    old_click_sum = 0.0
    no_click = 0.0
    no_click_sum = 0.0

    # treat all instances with the same predicted_ctr as coming from the
    # same bucket
    last_ctr = predicted_ctr[i_sorted[0]] + 1.0

    for i in range(len(predicted_ctr)):
        if last_ctr != predicted_ctr[i_sorted[i]]: 
            auc_temp += (click_sum+old_click_sum) * no_click / 2.0        
            old_click_sum = click_sum
            no_click = 0.0
            last_ctr = predicted_ctr[i_sorted[i]]
        no_click += num_impressions[i_sorted[i]] - num_clicks[i_sorted[i]]
        no_click_sum += num_impressions[i_sorted[i]] - num_clicks[i_sorted[i]]
        click_sum += num_clicks[i_sorted[i]]
    auc_temp += (click_sum+old_click_sum) * no_click / 2.0
    auc = auc_temp / (click_sum * no_click_sum)
    return auc

In [ ]:
solution = pd.read_csv('./data/KDD_Track2_solution.csv')

In [ ]:
auc = scoreClickAUC(solution['clicks'].tolist(), solution['impressions'].tolist(), y_pre_all)
auc

In [ ]:
import joblib
joblib.dump(y_pre_all,'./data/result/nb_withDrop_alpha0.7.pkl')